In [11]:
import bs4
import selenium
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import time
import warnings
import sys
import os
import requests
warnings.filterwarnings("ignore")

# Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
def search_amazon(product):
    url='https://www.amazon.in/'
    driver=webdriver.Chrome()
    driver.get(url)
    delay=10
    driver.maximize_window()
    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CLASS_NAME,'nav_a')))
    try:
        time.sleep(5)
        search_box = driver.find_element(By.ID,'twotabsearchtextbox')
        search_box.send_keys(product)  # Send product name to the search box
        search=driver.find_element(By.ID,'nav-search-submit-text')
        search.submit()
        time.sleep(5)
    except TimeoutException as e:
        print("No such element", {e})
        
# Ask the user for a product to search
product = input('Enter the name of a product: ')
search_amazon(product)

Enter the name of a product: Guitar


# In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
driver = webdriver.Chrome()

# ask user for the product to search
product = input("Enter the product to search: ")

# open the webpage
driver.get('https://www.amazon.in')

# find the search box, enter the product and submit
search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
search_box.send_keys(product)
search_box.submit()

# initialize lists to store the scraped data
product_url = []
product_name=[]
product_price=[]
product_brand=[]
product_delivery=[]
product_availability=[]
product_return=[]

# scrape the details of each product listed in the first 3 pages
for _ in range(3):
    # find all products on the page
    products = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
    
    for product in products:
        # get the details
        try:
            brand = product.find_element(By.XPATH, './/span[@class="a-size-base-plus a-color-base a-text-normal"]').text
        except NoSuchElementException:
            brand = '-'
        product_brand.append(brand)
        
        try:
            name = product.find_element(By.XPATH, './/h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]').text
        except NoSuchElementException:
            name = '-'
        product_name.append(name)
        
        try:
            price = product.find_element(By.XPATH, './/span[@class="a-price-whole"]').text
        except NoSuchElementException:
            price = '-'
        product_price.append(price)
        
        try:
            returns = product.find_element(By.XPATH,'.//span[@class="a-size-base s-underline-text"]').text
        except NoSuchElementException:
            returns = '-'
        product_return.append(returns)
        
        try:
            delivery = product.find_element(By.XPATH, './/span[@class="a-color-base a-text-bold"]').text
        except NoSuchElementException:
            delivery = '-'
        product_delivery.append(delivery)
        
        try:
            availability = product.find_element(By.XPATH, './/span[@class="a-size-base a-color-price"]').text
        except NoSuchElementException:
            availability = '-'
        product_availability.append(availability)
        
        url = product.find_element(By.XPATH, './/a[@class="a-link-normal s-no-outline"]').get_attribute('href')
        product_url.append(url)
    
    # go to the next page
    try:
        next_page = driver.find_element(By.XPATH, '//a[@title="Next Page"]')
        next_page.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(next_page))
    except NoSuchElementException:
        break  # if there are less than 3 pages, break the loop

driver.quit()

# save the data to a pandas DataFrame and then to a CSV file
df = pd.DataFrame({
    'Brand Name': product_brand,
    'Product Name': product_name,
    'Price': product_price,
    'Return/Exchange': product_return,
    'Expected Delivery': product_delivery,
    'Availability': product_availability,
    'Product URL': product_url
})
df.to_csv('amazon_products.csv', index=False)


Enter the product to search: Guitar


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [4]:
driver = webdriver.Chrome()

# The keywords to search for
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

for keyword in keywords:
    # Open the webpage
    driver.get('https://www.google.com/imghp')
    driver.maximize_window()

    # Find the search box, enter the keyword and submit
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(keyword)
    search_box.submit()

    # Wait for the search results to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.rg_i')))

    # Find the image elements
    images = driver.find_elements(By.CLASS_NAME, 'rg_i.Q4LuWd')[:10]

    # Create a directory to save the images
    os.makedirs(keyword, exist_ok=True)

    for i, image in enumerate(images):
        try:
            # Click on the image to load its src attribute
            time.sleep(3)
            image.click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'n3VNCb')))

            # Get the src attribute
            src = driver.find_element(By.CLASS_NAME, 'rg_i Q4LuWd').get_attribute("src")

            # Download the image
            response = requests.get(src, stream=True)
            with open(os.path.join(keyword, f'{i}.jpg'), 'wb') as out_file:
                out_file.write(response.content)
        except NoSuchElementException:
            print(f"Element not found for image {i} in keyword {keyword}")
        except Exception as e:
            print(f"An error occurred for image {i} in keyword {keyword}: {e}")

driver.quit()

An error occurred for image 0 in keyword fruits: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symbol) [0x00007FF627515E02]
	(No symbol) [0x00007FF6273D05AB]
	(No symbol) [0x00007FF62741175C]
	(No symbol) [0x00007FF6274118DC]
	(No symbol) [0x00007FF62744CBC7]
	(No symbol) [0x00007FF6274320EF]
	(No symbol) [0x00007FF62744AAA4]
	(No symbol) [0x00007FF627431E83]
	(No symbol) [0x00007FF62740670A]
	(No symbol) [0x00007FF627407964]
	GetHandleVerifier [0x00007FF627920AAB+3694587]
	GetHandleVerifier [0x00007FF62797728E+4048862]
	GetHandleVerifier [0x00007FF62796F173+4015811]
	GetHandleVerifier [0x00007FF6276447D6+695590]
	(No symbol) [0x00007FF627520CE8]
	(No symbol) [0x00007FF62751CF34]
	(No symbol) [0x00007FF62751D062]
	(No symbol) [0x00007FF62750D3A3]
	BaseThreadInitThunk [0x00007FF9B511257D+29]
	RtlUserThreadStart [0x00007FF9B67EAA78+40]

An error occurred for image 1 in keyword fruits: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symb

An error occurred for image 0 in keyword cars: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symbol) [0x00007FF627515E02]
	(No symbol) [0x00007FF6273D05AB]
	(No symbol) [0x00007FF62741175C]
	(No symbol) [0x00007FF6274118DC]
	(No symbol) [0x00007FF62744CBC7]
	(No symbol) [0x00007FF6274320EF]
	(No symbol) [0x00007FF62744AAA4]
	(No symbol) [0x00007FF627431E83]
	(No symbol) [0x00007FF62740670A]
	(No symbol) [0x00007FF627407964]
	GetHandleVerifier [0x00007FF627920AAB+3694587]
	GetHandleVerifier [0x00007FF62797728E+4048862]
	GetHandleVerifier [0x00007FF62796F173+4015811]
	GetHandleVerifier [0x00007FF6276447D6+695590]
	(No symbol) [0x00007FF627520CE8]
	(No symbol) [0x00007FF62751CF34]
	(No symbol) [0x00007FF62751D062]
	(No symbol) [0x00007FF62750D3A3]
	BaseThreadInitThunk [0x00007FF9B511257D+29]
	RtlUserThreadStart [0x00007FF9B67EAA78+40]

An error occurred for image 1 in keyword cars: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symbol) 

An error occurred for image 0 in keyword Machine Learning: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symbol) [0x00007FF627515E02]
	(No symbol) [0x00007FF6273D05AB]
	(No symbol) [0x00007FF62741175C]
	(No symbol) [0x00007FF6274118DC]
	(No symbol) [0x00007FF62744CBC7]
	(No symbol) [0x00007FF6274320EF]
	(No symbol) [0x00007FF62744AAA4]
	(No symbol) [0x00007FF627431E83]
	(No symbol) [0x00007FF62740670A]
	(No symbol) [0x00007FF627407964]
	GetHandleVerifier [0x00007FF627920AAB+3694587]
	GetHandleVerifier [0x00007FF62797728E+4048862]
	GetHandleVerifier [0x00007FF62796F173+4015811]
	GetHandleVerifier [0x00007FF6276447D6+695590]
	(No symbol) [0x00007FF627520CE8]
	(No symbol) [0x00007FF62751CF34]
	(No symbol) [0x00007FF62751D062]
	(No symbol) [0x00007FF62750D3A3]
	BaseThreadInitThunk [0x00007FF9B511257D+29]
	RtlUserThreadStart [0x00007FF9B67EAA78+40]

An error occurred for image 1 in keyword Machine Learning: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A8

An error occurred for image 0 in keyword Guitar: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symbol) [0x00007FF627515E02]
	(No symbol) [0x00007FF6273D05AB]
	(No symbol) [0x00007FF62741175C]
	(No symbol) [0x00007FF6274118DC]
	(No symbol) [0x00007FF62744CBC7]
	(No symbol) [0x00007FF6274320EF]
	(No symbol) [0x00007FF62744AAA4]
	(No symbol) [0x00007FF627431E83]
	(No symbol) [0x00007FF62740670A]
	(No symbol) [0x00007FF627407964]
	GetHandleVerifier [0x00007FF627920AAB+3694587]
	GetHandleVerifier [0x00007FF62797728E+4048862]
	GetHandleVerifier [0x00007FF62796F173+4015811]
	GetHandleVerifier [0x00007FF6276447D6+695590]
	(No symbol) [0x00007FF627520CE8]
	(No symbol) [0x00007FF62751CF34]
	(No symbol) [0x00007FF62751D062]
	(No symbol) [0x00007FF62750D3A3]
	BaseThreadInitThunk [0x00007FF9B511257D+29]
	RtlUserThreadStart [0x00007FF9B67EAA78+40]

An error occurred for image 1 in keyword Guitar: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symb

An error occurred for image 0 in keyword Cakes: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symbol) [0x00007FF627515E02]
	(No symbol) [0x00007FF6273D05AB]
	(No symbol) [0x00007FF62741175C]
	(No symbol) [0x00007FF6274118DC]
	(No symbol) [0x00007FF62744CBC7]
	(No symbol) [0x00007FF6274320EF]
	(No symbol) [0x00007FF62744AAA4]
	(No symbol) [0x00007FF627431E83]
	(No symbol) [0x00007FF62740670A]
	(No symbol) [0x00007FF627407964]
	GetHandleVerifier [0x00007FF627920AAB+3694587]
	GetHandleVerifier [0x00007FF62797728E+4048862]
	GetHandleVerifier [0x00007FF62796F173+4015811]
	GetHandleVerifier [0x00007FF6276447D6+695590]
	(No symbol) [0x00007FF627520CE8]
	(No symbol) [0x00007FF62751CF34]
	(No symbol) [0x00007FF62751D062]
	(No symbol) [0x00007FF62750D3A3]
	BaseThreadInitThunk [0x00007FF9B511257D+29]
	RtlUserThreadStart [0x00007FF9B67EAA78+40]

An error occurred for image 1 in keyword Cakes: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6275A82B2+55298]
	(No symbol

# Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [5]:
driver = webdriver.Chrome()

# the smartphone model to search for
smartphone_model = 'Oneplus Nord'

# open the webpage
driver.get('https://www.flipkart.com')
driver.maximize_window()
time.sleep(5)
driver.find_element(By.XPATH,'/html/body/div[3]/div/span').click()  # close login popup if exists

# find the search box, enter the smartphone model and submit
search_box = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div/div/div/div[1]/div/div[1]/div/div[1]/header/div[1]/div[2]/form/div/div/input')
time.sleep(3)
search_box.send_keys(smartphone_model)
search_box.submit()

# parse the search results page
soup = BeautifulSoup(driver.page_source, 'html.parser')

# find all product divs
products = soup.find_all('div', {'class': '_2kHMtA'})

# create a dataframe to store the product details
df = pd.DataFrame(columns=["Brand Name", "Smartphone name", "Colour", "RAM", "Storage(ROM)", "Primary Camera", "Secondary Camera", "Display Size", "Battery Capacity", "Price", "Product URL"])

# iterate over each product
for product in products:
    details = product.find('div', {'class': '_3pLy-c'}).text.split('|')
    url = 'https://www.flipkart.com' + product.find('a', {'class': '_1fQZEK'})['href']
    price = product.find('div', {'class': '_30jeq3 _1_WHN1'}).text

    # extract the product details
    brand_name = details[0].split(' ')[0] if len(details) > 0 else '-'
    smartphone_name = ' '.join(details[0].split(' ')[1:]) if len(details) > 0 else '-'
    colour = details[1] if len(details) > 1 else '-'
    ram = details[2] if len(details) > 2 else '-'
    rom = details[3] if len(details) > 3 else '-'
    primary_camera = details[4] if len(details) > 4 else '-'
    secondary_camera = details[5] if len(details) > 5 else '-'
    display_size = details[6] if len(details) > 6 else '-'
    battery_capacity = details[7] if len(details) > 7 else '-'

    # append the product details to the dataframe
    df = df.append({"Brand Name": brand_name, "Smartphone name": smartphone_name, "Colour": colour, "RAM": ram, "Storage(ROM)": rom, "Primary Camera": primary_camera, "Secondary Camera": secondary_camera, "Display Size": display_size, "Battery Capacity": battery_capacity, "Price": price, "Product URL": url}, ignore_index=True)

# save the dataframe to a CSV file
df.to_csv('flipkart_smartphones.csv', index=False)

driver.quit()


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [6]:
driver = webdriver.Chrome()

# the city to search for
city_name = 'Vadodara'

# open the webpage
driver.get('https://www.google.com/maps')

# find the search box, enter the city name and submit
search_box = driver.find_element(By.XPATH,'/html/body/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input')
search_box.send_keys(city_name)
search_box.submit()

# wait for the search results to load
time.sleep(5)

# get the current url
url = driver.current_url

# extract the latitude and longitude from the url
latitude, longitude = url.split('@')[1].split(',')[0:2]

print(f'Latitude: {latitude}, Longitude: {longitude}')

driver.quit()


Latitude: 23.0111528, Longitude: 72.6269321


# 6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [7]:
# Make a request to the website
r = requests.get("https://www.digit.in/top-products/best-business-laptops-india-42.html")
r.content

# Parse the HTML content
soup = BeautifulSoup(r.content, 'html.parser')

# Find the tables that contain the laptop details
tables = soup.find_all('div', {'class': 'TopNumbeHeading active sticky-footer'})

# Create a dataframe to store the laptop details
df = pd.DataFrame(columns=["Name", "OS", "Display", "Processor", "Memory", "Weight", "Dimension", "Graphics Processor"])

# Iterate over each table
for table in tables:
    name = table.find('div', {'class': 'TopNumbeHeading sticky-footer'}).text
    specs = table.find_all('div', {'class': 'value'})

    os = specs[0].text
    display = specs[1].text
    processor = specs[2].text
    memory = specs[3].text
    weight = specs[4].text
    dimension = specs[5].text
    graphics_processor = specs[6].text

    # Append the laptop details to the dataframe
    df = df.append({"Name": name, "OS": os, "Display": display, "Processor": processor, "Memory": memory, "Weight": weight, "Dimension": dimension, "Graphics Processor": graphics_processor}, ignore_index=True)

# Save the dataframe to a CSV file
df.to_csv('top_10_gaming_laptops.csv', index=False)


# Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [8]:
driver = webdriver.Chrome()

# open the webpage
driver.get('https://www.forbes.com/billionaires/')
driver.maximize_window()

# wait for the page to load
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))

# get the page source and parse it with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# find the table that contains the billionaire details
table = soup.find_all('div', {'class': 'Table_dataCell__2QCve'})[0]

# create a dataframe to store the billionaire details
df = pd.DataFrame(columns=["Rank", "Name", "Net worth", "Age", "Citizenship", "Source", "Industry"])

# iterate over each row in the table
for row in table.find_all('div', {'class': 'table-row'}):
    cols = row.find_all('div')

    # extract the billionaire details
    rank = cols[0].text
    name = cols[1].text
    net_worth = cols[2].text
    age = cols[3].text
    citizenship = cols[4].text
    source = cols[5].text
    industry = cols[6].text

    # append the billionaire details to the dataframe
    df = df.append({"Rank": rank, "Name": name, "Net worth": net_worth, "Age": age, "Citizenship": citizenship, "Source": source, "Industry": industry}, ignore_index=True)

# save the dataframe to a CSV file
df.to_csv('forbes_billionaires.csv', index=False)

driver.quit()


# 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [12]:
# Setup webdriver
driver = webdriver.Chrome()
driver.get('https://www.youtube.com/watch?v=FXs3WG7M-qk&ab_channel=SeleTraining')
driver.maximize_window()

time.sleep(5)

# Scroll down to load comments
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

# Wait for comments to load
time.sleep(5)

# Parse page source with BeautifulSoup
soup = bs(driver.page_source, 'html.parser')
driver.close()

# Find comments
comment_divs = soup.select('ytd-comment-renderer')

comments = []
for div in comment_divs:
    # Extract comment, upvotes and time
    comment = div.select_one('yt-formatted-string#content-text').get_text(strip=True)
    upvotes = div.select_one('span#vote-count-middle').get_text(strip=True)
    time_posted = div.select_one('yt-formatted-string.published-time-text').get_text(strip=True)
    
    comments.append((comment, upvotes, time_posted))

# Save first 500 comments to a DataFrame
df = pd.DataFrame(comments[:500], columns=['Comment', 'Upvotes', 'Time Posted'])

# Save DataFrame to a CSV file
df.to_csv('youtube_comments.csv', index=False)

# 9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [13]:
# Setup webdriver
driver = webdriver.Chrome()
driver.get('https://www.hostelworld.com/s?q=London&country=England&city=London&type=city&id=3&from=2023-12-15&to=2023-12-18&guests=2&page=1')

time.sleep(5)

# Parse page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()

# Find hostels
hostels = soup.select('div.property-card')

for hostel in hostels:
    # Extract details
    name = hostel.select_one('div.property-name')
    distance = hostel.select_one('div.property-distance')
    ratings = hostel.select_one('div.property-rating')
    total_reviews = hostel.select_one('div.review')
    overall_reviews = hostel.select_one('span.left-margin')
    privates_from_price = hostel.select_one('strong.current')
    dorms_from_price = hostel.select('strong.current')[1] if len(hostel.select('strong.current')) > 1 else None
    facilities = [facility.get_text(strip=True) for facility in hostel.select('div.property-badges')]
    property_description = hostel.select_one('span.discounted')

    # Check if elements exist before calling get_text
    name = name.get_text(strip=True) if name else None
    distance = distance.get_text(strip=True) if distance else None
    ratings = ratings.get_text(strip=True) if ratings else None
    total_reviews = total_reviews.get_text(strip=True) if total_reviews else None
    overall_reviews = overall_reviews.get_text(strip=True) if overall_reviews else None
    privates_from_price = privates_from_price.get_text(strip=True) if privates_from_price else None
    dorms_from_price = dorms_from_price.get_text(strip=True) if dorms_from_price else None
    property_description = property_description.get_text(strip=True) if property_description else None

    print(f'Name: {name}\nDistance: {distance}\nRatings: {ratings}\nTotal Reviews: {total_reviews}\nOverall Reviews: {overall_reviews}\nPrivates From Price: {privates_from_price}\nDorms From Price: {dorms_from_price}\nFacilities: {facilities}\nProperty Description: {property_description}\n---')


Name: St Christopher's Village
Distance: None
Ratings: 8.2Fabulous
Total Reviews: None
Overall Reviews: None
Privates From Price: None
Dorms From Price: None
Facilities: []
Property Description: None
---
Name: Palmers Lodge - Swiss Cottage
Distance: None
Ratings: 8.8Fabulous
Total Reviews: None
Overall Reviews: None
Privates From Price: None
Dorms From Price: None
Facilities: []
Property Description: None
---
Name: Wombat's City Hostel London
Distance: Hostel- 3.6km from city centre
Ratings: 9.0Superb(15562)
Total Reviews: (15562)
Overall Reviews: (15562)
Privates From Price: ₹16,800
Dorms From Price: ₹4,235
Facilities: ['']
Property Description: ₹21,000
---
Name: St Christopher's Village
Distance: Hostel- 1.8km from city centre
Ratings: 8.2Fabulous(12623)
Total Reviews: (12623)
Overall Reviews: (12623)
Privates From Price: ₹12,953
Dorms From Price: ₹3,550
Facilities: ['']
Property Description: ₹14,393
---
Name: Onefam Notting Hill
Distance: Hostel- 5.5km from city centre
Ratings: 9.6S

#                                                    END